# Parenthesization
* Optimal evaluation of associative expression $A[0]\bullet A[1] \bullet...A[n-1]$.
* 当A为矩阵时，便是 Matrix chain multiplication 问题 (or Matrix Chain Ordering Problem, MCOP)
## fully parenthesized matrix chain multiplication
* A product of matrices is fully parenthesized if it is either a <font color=red>single</font>  matrix or the product of <font color=red>two</font> fully parenthesized matrix products, surrounded by parentheses.   
    Ex.: $ABCD = (A(B(CD))) = (A((BC)D))=((AB)(CD)) = ((A(BC))D) = (((AB)C)D)$
 
## DP求解
 * 设矩阵$A_i$的大小为$p_{i}\times p_{i+1}$.
 * 设 $m(i,j)$为计算 $A_{i...j}$ (i.e. $A_iA_{i+1}...A_j$) 的标量乘法次数的最小值。
### 递归式   
     \begin{equation}
     m(i,j) =
         \begin{cases}
             \underset{i\le k < j}{\min} \{m(i, k) + m(k+1, j) + \text{ cost of multiplication } (A_i ... A_k) \text{ by }(A_{k+1} ... A_j)\},  & i< j \\
             0,  & i = j
          \end{cases}
     \end{equation}  
### 子问题个数，即不同的m(i,j)的个数:   
每一对满足  $ 0\le i \le j \le n-1 $的i,j组合对应一个唯一的子问题，则共有$1+2+...+n = \frac{n(n+1)}{2} \binom{n}{2} + n $，i.e. <font color = red> $O(n^2)$</font>个子问题.
### 子问题复杂度
cost per subproblem: $ O(j-i) = O(n) $   
(对于某一个k, $\text{ cost of multiplication } (A_i ... A_{k}) \text{ by }(A_{k+1} ... A_j)\} = p_ip_{k+1}p_{j+1}$; k共有j-i种选择)
### 算法运行时间
共有$ \Theta(n^2)$ 个子问题，每个子问题最多有n-1种考虑，故运行时间为<font color=red>$O(n^3)$</font>.

## Python implementation
* $p = [p_0,p_1,...,p_n]$ 记录了矩阵链$A_1A_2...A_n$中每个矩阵的大小：$A_i: p_i\times p_{i+1}$.
* 自底向上方法按矩阵链长度l递增的顺序来实现算法。ps: l即包含矩阵$A_i, A_j$的所有矩阵的个数。
* s记录最佳的m(i,j)的分割点k.

### From bottom to up

In [4]:
class table(object):
    def __init__(self, size):
        """size: tuple (#columns, #rows)"""
        ncol = size[0]; nrow = size[1]
        self.table = [[None]*nrow] * ncol
        
    def settable(self, m, n, value):
        self.table[m][n] = value

import numpy as np

inf = float('inf')
def MCOP(p):
    n = len(p) - 1
    m = np.asarray([[inf]*n]*n)
    s = np.asarray([[inf]*n]*n)
    
    for i in range(n):  # case: j=i
        m[i][i] = 0 
    for l in range(2, n+1): # case: j!= i; then length l>=2
        for i in range(n-l+1): #要保证矩阵链的长度为l，则对应的i可以的取值范围为 0~n-l
            j = i + l -1 # [i,j]长度为l，则j = i+l-1
            for k in range(i, j):
                q = m[i][k] + m[k+1][j] + p[i]*p[k+1]*p[j+1]
                if q < m[i][j]:
                    m[i][j] = q
                    s[i][j] = k
    return m,s

p = [30,35,15,5,10,20,25]

m, s = MCOP(p)
print('m =', m, '\n','s =\n', s)

m = [[     0.  15750.   7875.   9375.  11875.  15125.]
 [    inf      0.   2625.   4375.   7125.  10500.]
 [    inf     inf      0.    750.   2500.   5375.]
 [    inf     inf     inf      0.   1000.   3500.]
 [    inf     inf     inf     inf      0.   5000.]
 [    inf     inf     inf     inf     inf      0.]] 
 s =
 [[ inf   0.   0.   2.   2.   2.]
 [ inf  inf   1.   2.   2.   2.]
 [ inf  inf  inf   2.   2.   2.]
 [ inf  inf  inf  inf   3.   4.]
 [ inf  inf  inf  inf  inf   4.]
 [ inf  inf  inf  inf  inf  inf]]


### Memorized

In [8]:
import numpy as np

inf = float('inf')
def Memorized_MC(p):
    n = len(p) - 1
    m = np.asarray([[inf]*n]*n)
    s = np.asarray([[inf]*n]*n)
    return lookup_chain1(m,p,0,n-1,s)

def lookup_chain(m,p,i,j,s):
    if m[i][j] < inf:
        return m[i][j], s
    if i == j:
        m[i][j] = 0
    else:
        for k in range(i,j):
            q = lookup_chain(m,p,i,k,s)[0] + lookup_chain(m,p,k+1,j,s)[0] + p[i]*p[k+1]*p[j+1]
            if q < m[i][j]:
                m[i][j] = q
                s[i][j] = k
    return m[i][j],s


def lookup_chain1(m,p,i,j,s):
    if m[i][j] < inf:
        return (m, s)
    if i == j:
        m[i][j] = 0
    else:
        for k in range(i,j):
            q = lookup_chain1(m,p,i,k,s)[0][i][k] + lookup_chain1(m,p,k+1,j,s)[0][k+1][j] + p[i]*p[k+1]*p[j+1]
            if q < m[i][j]:
                m[i][j] = q
                s[i][j] = k
    return (m,s)



p = [30,35,15,5,10,20,25]

(m,s)= Memorized_MC(p)
print('m =', m,'\n', 's=', s)

m = [[     0.  15750.   7875.   9375.  11875.  15125.]
 [    inf      0.   2625.   4375.   7125.  10500.]
 [    inf     inf      0.    750.   2500.   5375.]
 [    inf     inf     inf      0.   1000.   3500.]
 [    inf     inf     inf     inf      0.   5000.]
 [    inf     inf     inf     inf     inf      0.]] 
 s= [[ inf   0.   0.   2.   2.   2.]
 [ inf  inf   1.   2.   2.   2.]
 [ inf  inf  inf   2.   2.   2.]
 [ inf  inf  inf  inf   3.   4.]
 [ inf  inf  inf  inf  inf   4.]
 [ inf  inf  inf  inf  inf  inf]]
